# Ch10-1. RNN(Recurrent Neural Network:순환신경망)

#### : 상태가 고정된 데이터를  다른 신경망과 다르게 자연어 처리나 음성인식처럼 순서가 있는
####  데이터를 처리하는데 장점
#### (예): 언어 모델링과 텍스트 생성, 음성인식, 이미지캡션생성, 자동번역(기계번역), 유전자,손글씨,
####  센서 감지한 데이터, 주가등의 배열(시계열 데이터)등

## 1. 손글씨 이미지를 RNN방식으로 학습하고 예측하기

In [1]:
# 실행시 반드시  한번에 실행하면 결과가 나옴

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('C:/JupyterSpace/datasets/mnist/data',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting C:/JupyterSpace/datasets/mnist/data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting C:/JupyterSpace/datasets/mnist/data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting C:/JupyterSpace/datasets/mnist/data\t10k-images-idx3-ubyte.gz
Extracting C:/JupyterSpace/datasets/mnist/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
learning_rate=0.001 #최적화함수에서 사용할 학습률
batch_size=128 # 한번에  학습할 데이터
total_epoch=30#전체 데이터를 학습할 총 횟수
n_step=28
n_hidden=128 #hidden layer의 뉴런 갯수
n_input=28 #입력값의 크기
n_class=10 

In [4]:
X=tf.placeholder(tf.float32,[None,n_step, n_input]) 
Y=tf.placeholder(tf.float32,[None, n_class]) 
W=tf.Variable(tf.random_normal([n_hidden, n_class]))
b=tf.Variable(tf.random_normal([n_class]))

In [5]:
# 학습에 사용할 cell 만들기
cell=tf.nn.rnn_cell.BasicRNNCell(n_hidden)
outputs, states=tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
outputs=tf.transpose(outputs, [1,0,2])
outputs=outputs[-1] #맨 뒤 cell
model=tf.matmul(outputs, W)+b

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer=tf.train.AdamOptimizer(learning_rate).minimize(cost)


Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [7]:
total_batch= int(mnist.train.num_examples/batch_size)
for epoch in range(total_epoch):
    total_cost=0
    for i in range(total_batch):
        batch_xs, batch_ys=mnist.train.next_batch(batch_size)
        batch_xs=batch_xs.reshape((batch_size, n_step, n_input))
        _, cost_val=sess.run([optimizer, cost], feed_dict={X:batch_xs, Y: batch_ys})
        total_cost += cost_val
    print('Epoch: ', '%04d' % (epoch+1), 'AVG_cost= ','{:3f}'.format(total_cost/total_batch))
print('최적화 완료')
                                  

Epoch:  0001 AVG_cost=  0.535948
Epoch:  0002 AVG_cost=  0.231215
Epoch:  0003 AVG_cost=  0.184375
Epoch:  0004 AVG_cost=  0.164486
Epoch:  0005 AVG_cost=  0.140351
Epoch:  0006 AVG_cost=  0.129118
Epoch:  0007 AVG_cost=  0.120985
Epoch:  0008 AVG_cost=  0.105632
Epoch:  0009 AVG_cost=  0.106657
Epoch:  0010 AVG_cost=  0.098873
Epoch:  0011 AVG_cost=  0.098441
Epoch:  0012 AVG_cost=  0.095422
Epoch:  0013 AVG_cost=  0.091318
Epoch:  0014 AVG_cost=  0.090181
Epoch:  0015 AVG_cost=  0.084918
Epoch:  0016 AVG_cost=  0.087048
Epoch:  0017 AVG_cost=  0.078332
Epoch:  0018 AVG_cost=  0.078100
Epoch:  0019 AVG_cost=  0.080706
Epoch:  0020 AVG_cost=  0.073791
Epoch:  0021 AVG_cost=  0.076635
Epoch:  0022 AVG_cost=  0.070303
Epoch:  0023 AVG_cost=  0.073728
Epoch:  0024 AVG_cost=  0.073505
Epoch:  0025 AVG_cost=  0.066089
Epoch:  0026 AVG_cost=  0.063754
Epoch:  0027 AVG_cost=  0.073691
Epoch:  0028 AVG_cost=  0.060882
Epoch:  0029 AVG_cost=  0.060952
Epoch:  0030 AVG_cost=  0.068430
최적화 완료


In [8]:
is_correct=tf.equal(tf.argmax(model,1), tf.argmax(Y,1))
accuracy=tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [9]:
test_batch_size=len(mnist.test.images)
test_xs=mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys=mnist.test.labels

print('정확도:', sess.run(accuracy, feed_dict={X:test_xs, Y:test_ys}))

정확도: 0.9768
